In [7]:
import pandas as pd
from tensorflow import keras as ks
import numpy as np
from tensorflow.keras import Input
import tensorflow as tf

In [8]:
train = pd.read_csv('data/split_data/train_stats.csv', index_col=0)
test = pd.read_csv('data/split_data/test_stats.csv', index_col=0)
train.head()

,tension_index_int,mode_int,std_int,mean_int,var_int,pnn50_int,RMSSD_int,ivr_int,vpr_int,papr_int,...,y90,y91,y92,y93,y94,y95,y96,y97,y98,y99
0,4.693159,755.102222,218.034485,729.56,29.885751,43.434343,295.930377,0.000071,8.533016e-07,0.000146,...,0,0,0,0,0,0,0,0,0,0
1,4.693159,755.102222,218.498749,730.44,29.913306,44.444444,280.249816,0.000071,8.533016e-07,0.000146,...,0,0,0,0,0,0,0,0,0,0
2,4.692843,755.152941,218.698723,731.00,29.917746,44.444444,294.181144,0.000071,8.532442e-07,0.000146,...,0,0,0,0,0,0,0,0,0,0
3,4.692843,755.152941,218.972510,731.64,29.928996,45.454545,294.238274,0.000071,8.532442e-07,0.000146,...,0,0,0,0,0,0,0,0,0,0
4,4.692843,755.152941,219.083457,731.88,29.934341,44.444444,294.196251,0.000071,8.532442e-07,0.000146,...,0,0,0,0,0,0,0,0,0,0


In [9]:
time_series_input = Input(shape=10)
full_time_series_input = Input(shape=10)
r_r_series_input = Input(shape=100)

time_series_layer = ks.layers.Dense(32, activation='tanh')(time_series_input)
time_series_layer = ks.layers.Dense(64, activation='tanh')(time_series_layer)
time_series_model = ks.Model(inputs=time_series_input, outputs=time_series_layer)

full_time_series_layer = ks.layers.Dense(32, activation='tanh')(full_time_series_input)
full_time_series_layer = ks.layers.Dense(64, activation='tanh')(full_time_series_layer)
full_time_series_model = ks.Model(inputs=full_time_series_input, outputs=full_time_series_layer)

r_r_series_layer = ks.layers.Dense(32, activation='tanh')(r_r_series_input)
r_r_series_layer = ks.layers.Dense(64, activation='tanh')(r_r_series_layer)
r_r_series_model = ks.Model(inputs=r_r_series_input, outputs=r_r_series_layer)

concatenate_layer = ks.layers.concatenate(inputs=[time_series_model.output,
                                                full_time_series_model.output,
                                                r_r_series_model.output])

concatenate_layer = ks.layers.Dense(64, 'tanh')(concatenate_layer)
concatenate_layer = ks.layers.Dense(32, 'tanh')(concatenate_layer)
concatenate_layer = ks.layers.Dense(100, 'softmax')(concatenate_layer)

model = ks.Model(inputs=[time_series_model.input,
                         full_time_series_model.input,
                         r_r_series_model.input,],
                 outputs=concatenate_layer)

In [10]:
def weighted_loss(y_true: tf.Tensor , y_pred: tf.Tensor):
    loss = ks.losses.BinaryCrossentropy()(y_true, y_pred)
    loss = loss * tf.constant([0.12, 0.88])
    return loss

In [11]:
model.compile(optimizer='adam', loss=weighted_loss, metrics='accuracy')


In [12]:
model.fit(x=[train.iloc[:, 10:20].values,
             train.iloc[:, :10].values,
             train.iloc[:, 20:-100].values,],
          y=train.iloc[:, -100:].values,
          batch_size=32,
          epochs=50)

Epoch 1/50
112/112 [==============================] - 4s 9ms/step - loss: 0.2580 - accuracy: 0.0062  

In [71]:
pred = model.predict([test.iloc[:, 10:20].values,
             test.iloc[:, :10].values,
             test.iloc[:, 20:-100].values,])


In [72]:
pred[pred > 0.01] = 1
pred[pred <= 0.01] = 0


In [73]:
from sklearn.metrics import confusion_matrix, f1_score

In [74]:
f1_score(y_true=test.iloc[:, -100:].values.reshape(1,-1)[0], y_pred=pred.reshape(1,-1)[0], average='micro')

0.6173321067157314

In [75]:
confusion_matrix(y_true=test.iloc[:, -100:].values.reshape(1,-1)[0], y_pred=pred.reshape(1,-1)[0])


array([[61264, 33553],
       [ 8043,  5840]], dtype=int64)

In [45]:
13883 / (94817 + 13883)

0.1277184912603496